In [1]:
import json
import logging
import os
from typing import List

import pandas as pd
from sentence_transformers import CrossEncoder

from financerag.rerank import CrossEncoderReranker
from financerag.retrieval import DenseRetrieval, SentenceTransformerEncoder, BM25Retriever, BM25Model
from financerag.tasks import (
    BaseTask,
    ConvFinQA,
    FinDER,
    FinQABench,
    FinQA,
    FinanceBench,
    MultiHiertt,
    TATQA
)
# Step 1: Import necessary libraries
# --------------------------------------
# Import required libraries for document retrieval, reranking, and logging setup.
from sentence_transformers import CrossEncoder
import logging

from financerag.rerank import CrossEncoderReranker, RRFReranker
from financerag.retrieval import DenseRetrieval, BM25Retriever, SentenceTransformerEncoder
from financerag.tasks import FinDER

# Setup basic logging configuration to show info level messages.
logging.basicConfig(level=logging.INFO)


from task_runner import TaskRunner


import gc
import torch

/FinanceRAG/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
all_tasks: List[type] = [
    ConvFinQA,
    FinDER,
    FinQABench,
    FinQA,
    FinanceBench,
    MultiHiertt,
    TATQA
]

In [ ]:
def run(task: BaseTask, results_dir: str = "./results"):
    print(f"Running {task.metadata.name}")
    encoder_model = SentenceTransformerEncoder(
        model_name_or_path='intfloat/e5-large-v2',
        query_prompt='query: ',
        doc_prompt='passage: ',
    )
    dense_retrieval_model = DenseRetrieval(
        model=encoder_model
    )

    bm25_model = BM25Model(task.corpus)
    sparse_model = BM25Retriever(
        model=bm25_model
    )

    retrieval_result = task.retrieve_hybrid(
        dense_retriever=dense_retrieval_model,
        sparse_retriever=sparse_model
    )

    print(f"Retrieved hybrid results for {len(retrieval_result)} queries. Here's an example of the top 5 documents for the first query:")

    for q_id, result in retrieval_result[0].items():
        print(f"\nQuery ID: {q_id}")
        # Sort the result to print the top 5 document ID and its score
        sorted_results = sorted(result.items(), key=lambda x: x[1], reverse=True)

        for i, (doc_id, score) in enumerate(sorted_results[:5]):
            print(f"  Document {i + 1}: Document ID = {doc_id}, Score = {score}")

        break  # Only show the first query

    for q_id, result in retrieval_result[1].items():
        print(f"\nQuery ID: {q_id}")
        # Sort the result to print the top 5 document ID and its score
        sorted_results = sorted(result.items(), key=lambda x: x[1], reverse=True)

        for i, (doc_id, score) in enumerate(sorted_results[:5]):
            print(f"  Document {i + 1}: Document ID = {doc_id}, Score = {score}")

        break  # Only show the first query

    # Delete references
    del encoder_model
    del dense_retrieval_model
    del bm25_model
    del sparse_model
    
    # Force garbage collection (CPU RAM)
    gc.collect()

    reranker = RRFReranker()
    
    
    # Clear GPU cache (Crucial for SentenceTransformer/PyTorch)
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    task.rerank(
        reranker=reranker,
        results=retrieval_result,
        top_k=100,  # Rerank the top 100 documents
        k=6
    )

    del reranker

    task.save_results(output_dir='./results')
    task.save_original_results(output_dir='./results')

In [4]:
for task in all_tasks:
    current_task: BaseTask = task(corpus_file='corpus_prep.jsonl', query_file='queries_prep.jsonl')
    TaskRunner.run_custom(current_task, run, results_dir='./results')
    evaluate_result = TaskRunner.evaluate(task, results_dir='./results')
    print(current_task.metadata.name)
    print(TaskRunner.format_results(evaluate_result))

INFO:financerag.common.loader:Loading Corpus...
INFO:financerag.common.loader:Loaded 2066 Documents.
INFO:financerag.common.loader:Corpus Example: {'id': 'dd4bff516', 'title': '', 'text': 'containerboard , kraft papers and saturating kraft .\nkapstone also owns victory packaging , a packaging solutions distribution company with facilities in the u.s. , canada and mexico .\nwe have included the financial results of kapstone in our corrugated packaging segment since the date of the acquisition .\non september 4 , 2018 , we completed the acquisition ( the 201cschl fcter acquisition 201d ) of schl fcter print pharma packaging ( 201cschl fcter 201d ) .\nschl fcter is a leading provider of differentiated paper and packaging solutions and a german-based supplier of a full range of leaflets and booklets .\nthe schl fcter acquisition allowed us to further enhance our pharmaceutical and automotive platform and expand our geographical footprint in europe to better serve our customers .\nwe have i

Running ConvFinQA


INFO:financerag.retrieval.dense:Encoding queries...
Batches: 100%|██████████| 7/7 [00:00<00:00,  7.98it/s]
INFO:financerag.retrieval.dense:Sorting corpus by document length...
INFO:financerag.retrieval.dense:Encoding corpus in batches... This may take a while.
INFO:financerag.retrieval.dense:Encoding batch 1/1...
Batches: 100%|██████████| 33/33 [00:43<00:00,  1.32s/it]
INFO:financerag.retrieval.bm25:Tokenizing queries with lower cases
INFO:financerag.tasks.BaseTask:Output directory set to: ./results/ConvFinQA
INFO:financerag.tasks.BaseTask:Saving top 10 results to CSV file: ./results/ConvFinQA/results.csv
INFO:financerag.tasks.BaseTask:Writing header ['query_id', 'corpus_id'] to CSV.
INFO:financerag.tasks.BaseTask:Top 10 results saved successfully to ./results/ConvFinQA/results.csv
INFO:financerag.tasks.BaseTask:Output directory set to: ./results/ConvFinQA
INFO:financerag.tasks.BaseTask:Saving original results to JSONL file: ./results/ConvFinQA/original_results.jsonl
INFO:financerag.ta

Retrieved hybrid results for 2 queries. Here's an example of the top 5 documents for the first query:

Query ID: qd4982518
  Document 1: Document ID = dd4b9f7f6, Score = 0.8448894619941711
  Document 2: Document ID = dd4b95a76, Score = 0.8383541107177734
  Document 3: Document ID = dd4c4c208, Score = 0.836830735206604
  Document 4: Document ID = dd4bbc202, Score = 0.8367441892623901
  Document 5: Document ID = dd4c4f7aa, Score = 0.8364241719245911

Query ID: qd4982518
  Document 1: Document ID = dd4982158, Score = 68.39888029898678
  Document 2: Document ID = dd496d5d2, Score = 67.16417497569476
  Document 3: Document ID = dd4be45d6, Score = 66.87313528496995
  Document 4: Document ID = dd4b87d18, Score = 66.75538979870812
  Document 5: Document ID = dd4b9f7f6, Score = 66.7295786426677
ConvFinQA
	NDCG@1: 0.23810     NDCG@5: 0.39528     NDCG@10: 0.43385
	MAP@1: 0.23810      MAP@5: 0.34987      MAP@10: 0.36581
	Recall@1: 0.23810   Recall@5: 0.53175   Recall@10: 0.65079
	P@1: 0.23810     

INFO:financerag.common.loader:Loaded 13867 Documents.
INFO:financerag.common.loader:Corpus Example: {'id': 'ADBE20230004', 'title': 'ADBE OVERVIEW', 'text': 'Adobe is a global technology company with a mission to change the world through personalized digital experiences. For over four decades, Adobe’s innovations have transformed how individuals, teams, businesses, enterprises, institutions, and governments engage and interact across all types of media. Our products, services and solutions are used around the world to imagine, create, manage, deliver, measure, optimize and engage with content across surfaces and fuel digital experiences. We have a diverse user base that includes consumers, communicators, creative professionals, developers, students, small and medium businesses and enterprises. We are also empowering creators by putting the power of artificial intelligence (“AI”) in their hands, and doing so in ways we believe are responsible. Our products and services help unleash crea

...

Retrieved hybrid results for 2 queries. Here's an example of the top 5 documents for the first query:

Query ID: q1a73c1d4
  Document 1: Document ID = d1b2ee16c, Score = 0.8175246119499207
  Document 2: Document ID = d1b3af812, Score = 0.8169326186180115
  Document 3: Document ID = d1b3576b2, Score = 0.8161620497703552
  Document 4: Document ID = d1b3aad30, Score = 0.8156298995018005
  Document 5: Document ID = d1b371530, Score = 0.8155505657196045

Query ID: q1a73c1d4
  Document 1: Document ID = d1b30ab0a, Score = 18.479142477631633
  Document 2: Document ID = d1b326d46, Score = 18.247789249761176
  Document 3: Document ID = d1b2f539a, Score = 18.182879631761047
  Document 4: Document ID = d1b3be84e, Score = 18.09071465674939
  Document 5: Document ID = d1b3bf28a, Score = 17.2841490621574


INFO:financerag.tasks.BaseTask:Output directory set to: ./results/TAT-QA
INFO:financerag.tasks.BaseTask:Saving top 10 results to CSV file: ./results/TAT-QA/results.csv
INFO:financerag.tasks.BaseTask:Writing header ['query_id', 'corpus_id'] to CSV.
INFO:financerag.tasks.BaseTask:Top 10 results saved successfully to ./results/TAT-QA/results.csv
INFO:financerag.tasks.BaseTask:Output directory set to: ./results/TAT-QA
INFO:financerag.tasks.BaseTask:Saving original results to JSONL file: ./results/TAT-QA/original_results.jsonl
INFO:financerag.tasks.BaseTask:Original results saved successfully to ./results/TAT-QA/original_results.jsonl
INFO:financerag.tasks.BaseTask:For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:NDCG@1: 0.3012
INFO:financerag.tasks.BaseTask:NDCG@5: 0.4184
INFO:financerag.tasks.BaseTask:NDCG@10: 0.4536
INFO:financerag.task

TAT-QA
	NDCG@1: 0.30120     NDCG@5: 0.41837     NDCG@10: 0.45355
	MAP@1: 0.30120      MAP@5: 0.38273      MAP@10: 0.39731
	Recall@1: 0.30120   Recall@5: 0.52610   Recall@10: 0.63454
	P@1: 0.30120        P@5: 0.10522        P@10: 0.06345



INFO:financerag.tasks.BaseTask:For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:NDCG@1: 0.1781
INFO:financerag.tasks.BaseTask:NDCG@5: 0.1144
INFO:financerag.tasks.BaseTask:NDCG@10: 0.1182
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:MAP@1: 0.0464
INFO:financerag.tasks.BaseTask:MAP@5: 0.0651
INFO:financerag.tasks.BaseTask:MAP@10: 0.0683
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:Recall@1: 0.0464
INFO:financerag.tasks.BaseTask:Recall@5: 0.0940
INFO:financerag.tasks.BaseTask:Recall@10: 0.1105
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:P@1: 0.1781
INFO:financerag.tasks.BaseTask:P@5: 0.0753
INFO:financerag.tasks.BaseTask:P@10: 0.0452
INFO:financerag.tasks.BaseTask:For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=Fa

-- query expansion + keyword extraction + hybrid --

ConvFinQA
	NDCG@1: 0.23810     NDCG@5: 0.39528     NDCG@10: 0.43385
	MAP@1: 0.23810      MAP@5: 0.34987      MAP@10: 0.36581
	Recall@1: 0.23810   Recall@5: 0.53175   Recall@10: 0.65079
	P@1: 0.23810        P@5: 0.10635        P@10: 0.06508

FinDER
	NDCG@1: 0.21875     NDCG@5: 0.33058     NDCG@10: 0.36456
	MAP@1: 0.17500      MAP@5: 0.27995      MAP@10: 0.29655
	Recall@1: 0.17500   Recall@5: 0.43594   Recall@10: 0.53021
	P@1: 0.21875        P@5: 0.11250        P@10: 0.07188

FinQABench
	NDCG@1: 0.73333     NDCG@5: 0.85515     NDCG@10: 0.86703
	MAP@1: 0.73333      MAP@5: 0.82778      MAP@10: 0.83333
	Recall@1: 0.73333   Recall@5: 0.93333   Recall@10: 0.96667
	P@1: 0.73333        P@5: 0.18667        P@10: 0.09667

FinQA
	NDCG@1: 0.22965     NDCG@5: 0.40279     NDCG@10: 0.44000
	MAP@1: 0.22965      MAP@5: 0.34966      MAP@10: 0.36533
	Recall@1: 0.22965   Recall@5: 0.56395   Recall@10: 0.67733
	P@1: 0.22965        P@5: 0.11279        P@10

In [5]:
final_str = TaskRunner.save_metrics(tasks=all_tasks, title='query expansion + keyword extraction + hybrid', results_dir='results')
print(final_str)
TaskRunner.combine_results(tasks=all_tasks, results_dir='results')

INFO:financerag.tasks.BaseTask:For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:NDCG@1: 0.2381
INFO:financerag.tasks.BaseTask:NDCG@5: 0.3953
INFO:financerag.tasks.BaseTask:NDCG@10: 0.4339
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:MAP@1: 0.2381
INFO:financerag.tasks.BaseTask:MAP@5: 0.3499
INFO:financerag.tasks.BaseTask:MAP@10: 0.3658
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:Recall@1: 0.2381
INFO:financerag.tasks.BaseTask:Recall@5: 0.5317
INFO:financerag.tasks.BaseTask:Recall@10: 0.6508
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:P@1: 0.2381
INFO:financerag.tasks.BaseTask:P@5: 0.1064
INFO:financerag.tasks.BaseTask:P@10: 0.0651
INFO:financerag.tasks.BaseTask:For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=Fa

-- query expansion + keyword extraction + hybrid --

ConvFinQA
	NDCG@1: 0.23810     NDCG@5: 0.39528     NDCG@10: 0.43385
	MAP@1: 0.23810      MAP@5: 0.34987      MAP@10: 0.36581
	Recall@1: 0.23810   Recall@5: 0.53175   Recall@10: 0.65079
	P@1: 0.23810        P@5: 0.10635        P@10: 0.06508

FinDER
	NDCG@1: 0.21875     NDCG@5: 0.33058     NDCG@10: 0.36456
	MAP@1: 0.17500      MAP@5: 0.27995      MAP@10: 0.29655
	Recall@1: 0.17500   Recall@5: 0.43594   Recall@10: 0.53021
	P@1: 0.21875        P@5: 0.11250        P@10: 0.07188

FinQABench
	NDCG@1: 0.73333     NDCG@5: 0.85515     NDCG@10: 0.86703
	MAP@1: 0.73333      MAP@5: 0.82778      MAP@10: 0.83333
	Recall@1: 0.73333   Recall@5: 0.93333   Recall@10: 0.96667
	P@1: 0.73333        P@5: 0.18667        P@10: 0.09667

FinQA
	NDCG@1: 0.22965     NDCG@5: 0.40279     NDCG@10: 0.44000
	MAP@1: 0.22965      MAP@5: 0.34966      MAP@10: 0.36533
	Recall@1: 0.22965   Recall@5: 0.56395   Recall@10: 0.67733
	P@1: 0.22965        P@5: 0.11279        P@10